In [1]:
file = open('E:\\final edits\\LSTM DATA.txt', encoding='utf8')
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

In [2]:
lines = file.readlines()
# lines2 = file1.readlines()

# Close the files
file.close()
# file1.close()

# Join the lines into a single string
df = ' '.join(lines)
# data1 = ' '.join(lines2)
# Combine the lines from the files into two strings
df = ' '.join(lines)
# data1 = ' '.join(lines2)

# Remove unwanted characters
df = df.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace('”', '')
# data1 = data1.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace('”', '')

# Concatenate the two strings into a single string
# df = data + ' ' + data1

# Additional cleaning using regex
df = re.sub(r'\ufeff', '', df)
df = re.sub(r'[“”"‘’]', '', df)

# Tokenize the combined text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([df])

# Check the vocabulary size
print(len(tokenizer.word_index))

7172


In [4]:
input_sequences = []

for sentence in df.split('. '):  # Adjust the split based on your sentence structure
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

import pickle
pickle.dump(tokenizer,open('token.pkl','wb'))       

max_len = max([len(x) for x in input_sequences])
max_len

padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')
x = padded_input_sequences[:, :-1]
y = padded_input_sequences[:, -1]

print(x.shape)
print(y.shape)
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=7173)

In [9]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=7173)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM,BatchNormalization

In [12]:
model = Sequential()
model.add(Embedding(input_dim=14268,output_dim=100,input_shape=(151,)))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dense(7173,activation='softmax'))

e:\final edits\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 151, 100)       │     1,426,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 151, 128)       │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 151, 128)       │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7173)           │       925,317 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,733,045 (10.43 MB)

 Trainable params: 2,732,789 (10.42 MB)

 Non-trainable params: 256 (1.00 KB)

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(x,y,epochs=50,batch_size=64)

In [ ]:
model.save('feedback.h5')